In [271]:
import pandas as pd
import numpy as np
import os
import re
import io
import functools
import math
from sklearn.cluster import *
import matplotlib.pyplot as plt

In [301]:
k = 3
l = math.ceil(math.log(k, 2))

w = 500

low = 0.01
high = 0.10

In [273]:
def standardize(word):
    word = word.lower()
    return word
    
def create_dict(text_documents):
    dictionary = []
    for doc in text_documents:
        lines = doc.split("\n")
        for line in lines:
            words = re.split('[<>"-\\/\[\]}{|_,.;:!? \t(){}]', line)
            for i, word in enumerate(words):
                if i >= w:
                    break
                word = standardize(word)
                if word == '':
                    continue
                if word not in dictionary:
                    dictionary.append(word)
    return dictionary

def text_features_extractor(text):
    features = dict(zip(dictionary, np.zeros(len(dictionary), dtype='int')))
    keys = features.keys()
    lines = text.split("\n")
    for line in lines:
        words = re.split('[,.;:!? \t]', line)
        for i, word in enumerate(words):
            if i >= w:
                    break
            word = standardize(word)
            if word in keys:
                features[word] += 1
    return list(features.values())

In [274]:
category_names = ['vijesti', 'sport', 'magazin']
dataset = 'modified.csv'
data = pd.read_csv(dataset)
num_articles = data.shape[0]
num_labels = data['category'].nunique()
labels = data['category']
labels_head = np.unique(labels.to_numpy())
dictionary = create_dict(data['modified'])

In [275]:
data

,url,title,content,category,modified
0,http://index.hr/magazin/clanak/be-proud-be-pow...,be-proud-be-powerful-be-pink,"ESSENCE pokreće kampanju brenda koja, između o...",magazin,kampànja brȇnda vȅčēr prihvaća rje násīlje ȁs1...
1,http://index.hr/magazin/clanak/bebek-1986-otis...,bebek-1986-otisao-sam-iz-dugmeta-jer-je-bregov...,GLAZBENIK Željko Bebek jučer je proslavio 76. ...,magazin,eb rȍđendān suràdnja ugopa njègov intèrvjū gȍd...
2,http://index.hr/magazin/clanak/beslagic-trola-...,beslagic-trola-politicare-ovo-su-tri-konstitut...,ENIS Bešlagić je na Instagramu objavio video s...,magazin,ešl nstag vȋdeo jed ožićn Pȃd švéma tróla polì...
3,http://index.hr/magazin/clanak/cedo-komljenovi...,cedo-komljenovic-decko-iz-karlovca-koji-je-ost...,DANAS je u Milanu u 83. godini umro Čedo Komlj...,magazin,il gȍdina čȅdo omlje on ha pȍduzētnīk mènadžer...
4,http://index.hr/magazin/clanak/ekipa-sa-stojed...,ekipa-sa-stojedinice-pokrece-novi-radio-da-je-...,PRESTANAK emitiranja programa Radija 101 ostav...,magazin,emiti prògram ad praznìna éter tojed póstojānj...
...,...,...,...,...,...
485,https://web.archive.org/web/20210719095306/htt...,umro-danski-karikaturist-poznat-po-crtezu-pror...,"DANSKI karikaturist Kurt Westergaard, slavan z...",vijesti,karikatùrist urt ester zbog kar uha nȅrēd zèml...
486,https://web.archive.org/web/20210719095306/htt...,vandjelic-iz-ministarstva-nije-dosla-nijedna-o...,RAVNATELJ Fonda za obnovu Damir Vanđelić izjav...,vijesti,ȍbnova ámir ȃnđeo ond ȍbnova sȁv dȍk2 pròces d...
487,https://web.archive.org/web/20210719095306/htt...,video-u-beogradu-se-djelomicno-urusila-stamben...,DANAS se u beogradskom naselju Vračar djelomič...,vijesti,násēlje rȁca zgrȁda mȇdīj kȁt1 stànār idovdans...
488,https://web.archive.org/web/20210719095306/htt...,vujcic-opet-imamo-stari-problem-nedostaje-kval...,GUVERNER Hrvatske narodne banke (HNB) Boris Vu...,vijesti,rva bànka òris ȕjāk ubrov da tròmjesēčje stò...


In [276]:
len(data)

490

In [277]:
#index
A = data['modified'].apply(lambda row: text_features_extractor(row)).values.tolist()
A = np.transpose(A)
# print(A)

In [278]:
A = np.array(A)
print(A.shape)

(10224, 490)


In [279]:
# filtracija A, izbacivanje rijetkih i cestih rijeci
df = pd.DataFrame(A)
df2 = df.applymap(lambda x: 1 if x != 0 else 0)
counts = list(df2.apply(lambda x: sum(x) / len(labels), axis=1))
selected = [low < count < high for count in counts]
df['words'] = dictionary
df = df.loc[selected, :]
wordsmap = df['words']
wordsmap = wordsmap.reset_index()
df = df.drop(['words'], axis = 1)
A = df.to_numpy()
#print(A)

In [280]:
wordsmap

,index,words
0,0,kampànja
1,2,vȅčēr
2,5,násīlje
3,6,ȁs1
4,7,bòja
...,...,...
1748,8559,rȕpa
1749,8571,čimpànza
1750,8684,vatrogásac
1751,8729,popuš


In [281]:
print(A.shape)

(1753, 490)


In [302]:
D1_diag = np.sum(A, axis=1)
D1 = np.diag(D1_diag)

In [303]:
D2_diag = np.sum(A, axis=0)
D2 = np.diag(D2_diag)

In [304]:
D1_inv = np.diag(list(map(lambda x: 0 if x == 0 else 1/(np.sqrt(x)), D1_diag)))
D2_inv = np.diag(list(map(lambda x: 0 if x == 0 else 1/(np.sqrt(x)), D2_diag)))

In [305]:
An = D1_inv @ A @ D2_inv

In [306]:
u, s, vt = np.linalg.svd(An)
v = np.transpose(vt)

In [307]:
U = u[:, 1:(1+l)]
V = v[:, 1:(1+l)]
print(U.shape)
print(V.shape)

(1753, 2)
(490, 2)


In [308]:
Z = np.block([[D1_inv @ U], [D2_inv @ V]])

In [309]:
kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300, random_state=42)
kmeans.fit(Z)
predictions = kmeans.predict(Z)

In [310]:
table = np.zeros((k, num_labels), dtype='int')
table = pd.DataFrame(table, columns=labels_head)


for i, pred in enumerate(predictions[-num_articles:]):
    label = labels.iloc[i]
#     print("{}-{}".format(pred, label))
    table[label][pred] += 1
print(table)

   magazin  sport  vijesti
0       16    176        0
1       92      5      160
2        1      3       37


In [311]:
predictions_words = predictions[0:-num_articles]
predictions_docs = predictions[-num_articles:]

In [312]:
for i in range(k):
    print('klaster {}'.format(i))
    selected = predictions_docs == i
    B = pd.DataFrame(A)
    B = B.iloc[:, selected]
    B['weight'] = B.sum(axis = 1, skipna = True)
    B_sorted = B.sort_values(by='weight', ascending=False)
    print(wordsmap.iloc[B_sorted.index[0:10]])

klaster 0
      index           words
159     373  reprezentácija
1321   4505             mȅč
1364   4650          tréner
1057   2946           pẕvāk
349     738            èuro
1398   4758           arcel
1329   4530           mòmče
858    2144         mèdalja
540    1175              am
981    2556          tùrnīr
klaster 1
      index     words
429     901      kúna
267     581    tvȑtka
682    1592        rb
1438   4887  polìcija
660    1542    dijéte
419     882    cijéna
539    1174         –
4         7      bòja
630    1481     nòvac
290     621      žèna
klaster 2
      index      words
1185   3523    cjȅpīvo
860    2146       dóza
347     731   cijeplje
1117   3181        cov
1497   5223      vírus
614    1411  varìjanta
1646   6898      dȅlta
725    1708     bȍlēst
956    2467      ritan
845    2091  infèkcija


In [248]:
B = pd.DataFrame(A)
B=B.drop(B.columns[predictions_docs==0], axis = 1)
B['weight'] = B.sum(axis = 1, skipna = True)
B_sorted = B.sort_values(by='weight', ascending=False)
wordsmap.iloc[B_sorted.index[0:10]]

,index,words
1185,3523,cjȅpīvo
860,2146,dóza
347,731,cijeplje
429,901,kúna
349,738,èuro
682,1592,rb
267,581,tvȑtka
1117,3181,cov
343,727,mjȅra
539,1174,–


In [30]:
predictions_words

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [31]:
predictions_docs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [313]:
num_articles

490